# **Atividade Computacional 03 - TEEC III**
## **Aluno: André Paiva Conrado Rodrigues**

## Importando bibliotecas

Nesta seção do código, são importadas bibliotecas de manipulação de dados importantes para a obtenção das amostras dos arquivos de áudio, bem como para a manipulação das mesmas.

In [ ]:
import numpy as np
import math
from scipy import signal
from scipy.io import wavfile
from IPython.display import Audio
import matplotlib.pyplot as plt

## Função para plotagem de espectrograma

In [ ]:
def plotar(dataArray, title):
    plt.figure(figsize=(17,5))
    plt.margins(0, 0)    
    plt.xlabel('Amostras')
    plt.title(title)
    plt.plot(dataArray)
    plt.show()

In [ ]:
def spectroPlot(audioArray, rate, windowSize, title):
    f, t, Sxx = signal.spectrogram(audioArray, rate, nperseg=windowSize)
    plt.figure(figsize=(17,5))
    plt.pcolormesh(t, f, np.abs(Sxx), shading='auto', cmap='bone', vmin=0, vmax=4095)
    plt.title(title)
    plt.ylabel('Frequência [Hz]')
    plt.xlabel('Tempo [s]')
    plt.colorbar()
    plt.show()

In [ ]:
def resampleBy2(filename, windowSize, resampleType):
    samplerate, data = wavfile.read(filename)
    qtdSamples = data.shape[0]
    resampleList = []
    if resampleType == "up":
        resampleRate = int(samplerate*2)
    elif resampleType == "down":
        resampleRate = int(samplerate/2)
    i = 0
    while(i < qtdSamples):
        resampleList.append(data[i])
        if resampleType == "up":
            resampleList.append(0)
            i = i + 1
        elif resampleType == "down":
            i = i + 2
    resampleArray = np.array(resampleList)
    del resampleList
    if resampleType == "up":
        sos = signal.butter(2, samplerate/2, 'lp', fs=resampleRate, output='sos')
        resampleArray = signal.sosfilt(sos, resampleArray)
    title = " - Áudio " + resampleType + "sampled"
    print("Player - Áudio original")
    display(Audio(filename, rate=samplerate))
    print("Player" + title)
    display(Audio(resampleArray, rate=resampleRate))
    spectroPlot(data, samplerate, windowSize, "Espectrograma - Áudio original")
    spectroPlot(resampleArray, resampleRate, windowSize, "Espectrograma" + title)
    plotar(data, "Waveform - Áudio original")
    plotar(resampleArray, "Waveform" + title)

In [ ]:
resampleBy2('highfreq.wav', 64, 'up')
resampleBy2('highfreq.wav', 64, 'down')